In [ ]:
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
import matplotlib as mpl
import scipy.stats as st
from scipy.stats import (vonmises_line, norm, t, cauchy, lognorm, fisk, skewnorm, exponweib, genlogistic, logistic, laplace)
import hdbscan
import seaborn as sns
import colorcet as cc
import astropy.units as u
import xarray as xr
from sunpy.visualization.colormaps import cm
import zarr
import sunpy.map
from sunpy.coordinates import HeliographicCarrington
from astropy.coordinates import SkyCoord
from sunpy.map.header_helper import make_heliographic_header
from sunpy.coordinates import frames

from scipy import stats

import random

import glob
import os
from tqdm.notebook import tqdm

### Define colors an colormaps

In [ ]:
# Color definitions
ClrS = (0.74, 0.00, 0.00)
ClrN = (0.20, 0.56, 1.00)

Clr = [(0.00, 0.00, 0.00),
      (0.31, 0.24, 0.00),
      (0.43, 0.16, 0.49),
      (0.32, 0.70, 0.30),
      (0.45, 0.70, 0.90),
      (1.00, 0.82, 0.67)]

### Read data, add date, Calibrate flux for MDI using 0.8 factor, and apply 3x10^21 Mx cuttoff

In [ ]:
# Flux Cutoff in Maxwells
fCutoff = 3e21

KPVT = pd.read_csv('data/output/BARD_1976-1993_KPVT_tilts.csv')
KPVT = KPVT.loc[KPVT['BMRFlux']>=fCutoff,:]
KPVT['Date'] = pd.to_datetime(dict(year=KPVT.Year, month=KPVT.Month, day=KPVT.Day, hour=KPVT.Hour, minute=KPVT.Minute))
KPVT['Instrument'] = 'KPVT'

SPMG = pd.read_csv('data/output/BARD_1992-1999_SPMG_tilts.csv')
SPMG = SPMG.loc[SPMG['BMRFlux']>=fCutoff,:]
SPMG['Date'] = pd.to_datetime(dict(year=SPMG.Year, month=SPMG.Month, day=SPMG.Day, hour=SPMG.Hour, minute=SPMG.Minute))
SPMG['Instrument'] = 'SPMG'
SPMG['BMRLabel'] = SPMG['BMRLabel'] + np.max(KPVT['BMRLabel'])

MDIcalibration = 0.8
MDI = pd.read_csv('data/output/BARD_1996-2010_MDI_tilts.csv')
MDI.loc[:, ['PFlux', 'NFlux', 'BMRFlux']] *= MDIcalibration
MDI = MDI.loc[MDI['BMRFlux']>=fCutoff,:]
MDI['Date'] = pd.to_datetime(dict(year=MDI.Year, month=MDI.Month, day=MDI.Day, hour=MDI.Hour, minute=MDI.Minute))
MDI['Instrument'] = 'MDI'
MDI['BMRLabel'] = MDI['BMRLabel'] + np.max(SPMG['BMRLabel'])

HMI = pd.read_csv('data/output/BARD_2010-2016_HMI_tilts.csv')
HMI = HMI.loc[HMI['BMRFlux']>=fCutoff,:]
HMI['Date'] = pd.to_datetime(dict(year=HMI.Year, month=HMI.Month, day=HMI.Day, hour=HMI.Hour, minute=HMI.Minute))
HMI['Instrument'] = 'HMI'
HMI['BMRLabel'] = HMI['BMRLabel'] + np.max(MDI['BMRLabel'])

print(np.max(HMI['BMRLongitude']), np.min(HMI['BMRLongitude']))


In [ ]:
HMI

### Remove overlaps between missions

In [ ]:
KPVT = KPVT.loc[KPVT['Date']<np.min(SPMG['Date']),:]
SPMG = SPMG.loc[SPMG['Date']<np.min(MDI['Date']),:]
MDI = MDI.loc[MDI['Date']<np.min(HMI['Date']),:]

### Concatenate all instruments

In [ ]:
allMag = pd.concat([KPVT, SPMG, MDI, HMI], ignore_index=True).drop(labels=['ReferenceDay', 'Flux_cut'], axis=1).replace(np.nan, False).sort_values(by = ['Date']).reset_index(drop = True)
allMag.to_csv('data/output/BARD_1976-2016_Merge_tilts.csv', index=False)

allMag['pltSize'] = (np.sqrt(allMag['BMRFlux']/1e21))
allMag

In [ ]:
for k in allMag.BMRLabel:
    df_k = pd.DataFrame((allMag.BMRLabel==k))*1
    
allMag[allMag.BMRLabel==247].index[1]
#allMag.drop(135,axis=0)

In [ ]:
for k in allMag.BMRLabel:
    df_k = (allMag.BMRLabel==k)*1
    suma = df_k.sum()#[0]
    if suma > 1:
        index_1 = allMag[allMag.BMRLabel==k].index[1]
        allMag = allMag.drop(index_1, axis=0)

In [ ]:
df = pd.DataFrame(allMag.BMRLabel)
counter = 0
for k in allMag.BMRLabel:
    df_k = (allMag.BMRLabel==k)*1
    suma = df_k.sum()#[0]
    if suma > 1:
        counter +=1
        print('Problema en BMRLabel ', k)

In [ ]:
allMag = allMag.reset_index()

In [ ]:
allMag.loc[allMag['BMRLongitude']<0, 'BMRLongitude'] = allMag.loc[allMag['BMRLongitude']<0, 'BMRLongitude']+360

### Plot data

In [ ]:
# Size definitions
dpi = 300
pxx = 1600  # Horizontal size of each panel
pxy = 500  # Vertical size of each panel

nph = 5     # Number of horizontal panels
npv = 6     # Number of vertical panels 

# Padding
padv  = 0  #Vertical padding in pixels
padv2 = 0  #Vertical padding in pixels between panels
padh  = 0 #Horizontal padding in pixels at the edge of the figure
padh2 = 0  #Horizontal padding in pixels between panels

# Figure sizes in pixels
fszv = (npv*pxy + 2*padv + (npv-1)*padv2 )      #Vertical size of figure in pixels
fszh = (nph*pxx + 2*padh + (nph-1)*padh2 )      #Horizontal size of figure in pixels

# Conversion to relative units
ppxx   = pxx/fszh      # Horizontal size of each panel in relative units
ppxy   = pxy/fszv      # Vertical size of each panel in relative units
ppadv  = padv/fszv     #Vertical padding in relative units
ppadv2 = padv2/fszv    #Vertical padding in relative units
ppadh  = padh/fszh     #Horizontal padding the edge of the figure in relative units
ppadh2 = padh2/fszh    #Horizontal padding between panels in relative units


## Start Figure
fig = plt.figure(figsize=(fszh/dpi,fszv/dpi), dpi = dpi)

ax1 = fig.add_axes([ppadh, ppadv, ppxx, ppxy])

size = (np.sqrt(KPVT['BMRFlux']/1e21))
ax1.scatter(KPVT['Date'], KPVT['BMRLatitude'], s=size, alpha=0.4, ec='None', label='KPVT/512')


size = (np.sqrt(SPMG['BMRFlux']/1e21))
ax1.scatter(SPMG['Date'], SPMG['BMRLatitude'], s=size, alpha=0.4, ec='None', label='KPVT/SPMG')


size = (np.sqrt(MDI['BMRFlux']/1e21))
ax1.scatter(MDI['Date'], MDI['BMRLatitude'], s=size, alpha=0.4, ec='None', label='SOHO/MDI')


size = (np.sqrt(HMI['BMRFlux']/1e21))
ax1.scatter(HMI['Date'], HMI['BMRLatitude'], s=size, alpha=0.4, ec='None', label='SDO/HMI')


size = (np.sqrt(allMag['BMRFlux']/1e21))
mask = allMag['AntiHale']
ax1.scatter(allMag.loc[mask,'Date'], allMag.loc[mask,'BMRLatitude'], s=size[mask], alpha=1,fc='k', ec='None')


ax1.set_ylim([-45, 45])
ax1.set_ylabel('Latitude (°)')
ax1.set_xlabel('Year')
ax1.legend(frameon=False, ncol=4, loc='upper center', bbox_to_anchor=(0.475, 1.2))



# BMR Dipole moment

In [ ]:
allMag['BMRColatitude'] = (90 - allMag.BMRLatitude) * np.pi/180
allMag['PColatitude'] = (90 - allMag.PLatitude) * np.pi/180
allMag['NColatitude'] = (90 - allMag.NLatitude) * np.pi/180

R_sun = 6.96 * 10e8 * 10e2  # cm
allMag['D_bmr'] = (3/(4*np.pi)) * allMag.BMRFlux * (np.cos(allMag.PColatitude) - np.cos(allMag.NColatitude)) / R_sun**2

In [ ]:
tbins = np.arange(-90, 90, 2)
x = np.linspace(-90, 90, 100)

# Size definitions
dpi = 400
pxx = 600  # Horizontal size of each panel
pxy = 500  # Vertical size of each panel

nph = 5     # Number of horizontal panels
npv = 6     # Number of vertical panels 

# Padding
padv  = 0  #Vertical padding in pixels
padv2 = 0  #Vertical padding in pixels between panels
padh  = 0 #Horizontal padding in pixels at the edge of the figure
padh2 = 0  #Horizontal padding in pixels between panels

# Figure sizes in pixels
fszv = (npv*pxy + 2*padv + (npv-1)*padv2 )      #Vertical size of figure in pixels
fszh = (nph*pxx + 2*padh + (nph-1)*padh2 )      #Horizontal size of figure in pixels

# Conversion to relative units
ppxx   = pxx/fszh      # Horizontal size of each panel in relative units
ppxy   = pxy/fszv      # Vertical size of each panel in relative units
ppadv  = padv/fszv     #Vertical padding in relative units
ppadv2 = padv2/fszv    #Vertical padding in relative units
ppadh  = padh/fszh     #Horizontal padding the edge of the figure in relative units
ppadh2 = padh2/fszh    #Horizontal padding between panels in relative units


# Combine hemispheres 


## Start Figure
fig = plt.figure(figsize=(fszh/dpi,fszv/dpi), dpi = dpi)

ax1 = fig.add_axes([ppadh, ppadv, 4*ppxx, ppxy])

size = (np.sqrt(allMag['BMRFlux']/1e21))


for cyc in np.arange(21,25):

    mask = np.logical_or(allMag[f'Cycle{str(cyc)}n'],allMag[f'Cycle{str(cyc)}s'])

    

    if cyc%2 == 0:

        maskp = np.logical_and(mask, allMag['D_bmr']>0)
        ax1.scatter(allMag.loc[maskp,'Date'], allMag.loc[maskp,'BMRLatitude'], s=size[maskp], alpha=0.4, ec='None', c='r')

        maskn = np.logical_and(mask, allMag['D_bmr']<0)
        ax1.scatter(allMag.loc[maskn,'Date'], allMag.loc[maskn,'BMRLatitude'], s=size[maskn], alpha=0.4, ec='None', c='b')

    else:

        maskn = np.logical_and(mask, allMag['D_bmr']<0)
        ax1.scatter(allMag.loc[maskn,'Date'], allMag.loc[maskn,'BMRLatitude'], s=size[maskn], alpha=0.4, ec='None', c='b')        

        maskp = np.logical_and(mask, allMag['D_bmr']>0)
        ax1.scatter(allMag.loc[maskp,'Date'], allMag.loc[maskp,'BMRLatitude'], s=size[maskp], alpha=0.4, ec='None', c='r')

    print(cyc, np.sum(allMag.loc[maskp, 'D_bmr']), np.sum(allMag.loc[maskn, 'D_bmr']))



ax1.set_ylim([-45, 45])
ax1.set_ylabel('Latitude (°)')
ax1.set_xlabel('Year')
# ax1.legend(frameon=False, ncol=5, loc='upper center', bbox_to_anchor=(0.475, 1.2))

### Remove spotless days

In [ ]:
allMag = allMag.loc[np.isfinite(allMag["BMRLatitude"]), :]

# Active longitudes

In [ ]:
# reference day
# time_0 = '1970-01-01 00:00:00'

time_0 = pd.to_datetime(0)
ref_day = (allMag['Date'] - time_0)

for i in range(len(ref_day)):
    ref_day[i] = ref_day[i].days

allMag['ReferenceDay'] = ref_day
ref_day

In [ ]:
# diferential rotation

def dif_rotation(lat, remove_carrington=False):
    A = 14.113463 *u.deg / u.d             # degrees/days
    B = -1.6979719 *u.deg / u.d            # degrees/days
    C = -1.787 *u.deg / u.d            # degrees/days
    c_r = 13.721173 *u.deg / u.d      # degrees/days

    if remove_carrington:
        return (A + B * np.sin(lat)**2 + C * np.sin(lat)**4) - c_r
    else:
        return (A + B * np.sin(lat)**2 + C * np.sin(lat)**4)

In [ ]:
dif_rotation(26*u.deg, remove_carrington=True)

In [ ]:
allMag["DRot"] = dif_rotation(allMag["BMRLatitude"].to_numpy()*u.deg, remove_carrington=True) #- 13.194485832888454 *u.deg / u.d

In [ ]:
plt.scatter(allMag["BMRLatitude"], allMag["DRot"])

In [ ]:
# distancia para una región
# https://www.movable-type.co.uk/scripts/latlong.html

def reg_distance(all_mag, region):
    # https://www.movable-type.co.uk/scripts/latlong.html
    
    lon = region['BMRLongitude']*u.deg
    lat = region['BMRLatitude']*u.deg
    ref_day = region['Date']

    rotation = dif_rotation(lat)
    
    bmr_latitude = all_mag['BMRLatitude'].values *u.deg
    bmr_longitude = all_mag['BMRLongitude'].values *u.deg

    rotation_diff = np.abs(dif_rotation(bmr_latitude) - rotation)

    time_diff = (((ref_day - all_mag['Date']).dt.total_seconds().values*u.s).to(u.d)).value
    corrected_longitude = bmr_longitude - rotation_diff * time_diff * u.d


    # distance

    R_sun = 695700 *u.km  # sun radius (km)
    delta_lon = corrected_longitude - lon

    delta_lat = (bmr_latitude - lat)
    harversin = np.power(np.sin(delta_lat/2), 2) + np.cos(lat)*np.cos(bmr_latitude)*np.power(np.sin(delta_lon/2), 2)
    delta_sigma = 2*np.arctan2(np.sqrt(harversin), np.sqrt(1-harversin))

    distance = delta_sigma.to(u.deg) # *R_sun

    delta_lon = delta_lon - (np.abs(delta_lon)//(360*u.deg)*np.sign(delta_lon.value))*360*u.deg
    delta_lon[delta_lon < -180*u.deg] = 360*u.deg+delta_lon[delta_lon < -180*u.deg]
    delta_lon[delta_lon > 180*u.deg] = -(360*u.deg-delta_lon[delta_lon > 180*u.deg])

    return (distance, np.abs(time_diff), np.abs(delta_lat), np.abs(delta_lon))    

#### Symmetry test

In [ ]:
## Pick two random regions
inx1 = np.random.randint(0, allMag.shape[0])
inx2 = np.random.randint(0, allMag.shape[0])

print(allMag.iloc[[inx1], 0:].loc[:,['BMRLatitude', 'BMRLongitude', 'Date']])
print(allMag.iloc[[inx2], 0:].loc[:,['BMRLatitude', 'BMRLongitude', 'Date']])

print(reg_distance(allMag.iloc[[inx2], 0:], allMag.iloc[inx1, 0:]))
print(reg_distance(allMag.iloc[[inx1], 0:], allMag.iloc[inx2, 0:]))

In [ ]:
## Pick first two regions
inx1 = 1
inx2 = 0

print(allMag.iloc[[inx1], 0:].loc[:,['BMRLatitude', 'BMRLongitude', 'Date']])
print(allMag.iloc[[inx2], 0:].loc[:,['BMRLatitude', 'BMRLongitude', 'Date']])

print(reg_distance(allMag.iloc[[inx2], 0:], allMag.iloc[inx1, 0:]))
print(reg_distance(allMag.iloc[[inx1], 0:], allMag.iloc[inx2, 0:]))

#### Test matrix with only a small subset

In [ ]:
shortMag = allMag.iloc[range(4),:].copy()

distance_matrix = np.zeros((len(shortMag), len(shortMag)))
time_diff_matrix = np.zeros((len(shortMag), len(shortMag)))

for k in range(len(shortMag)):
    region = shortMag.iloc[k,0:]
    distance = reg_distance(shortMag, region)#.to_numpy()
    distance_matrix[k,:] = distance[0]
    time_diff_matrix[k,:] = distance[1]

distance_matrix

In [ ]:
distance_matrix.T

In [ ]:
np.sum(distance_matrix - distance_matrix.T)

### Calculate distance

In [ ]:
distance_matrix = np.zeros((len(allMag), len(allMag)))
time_diff_matrix = np.zeros((len(allMag), len(allMag)))
lat_diff_matrix = np.zeros((len(allMag), len(allMag)))
lon_diff_matrix = np.zeros((len(allMag), len(allMag)))

for k in range(len(allMag)):
    region = allMag.iloc[k,0:]
    distance = reg_distance(allMag, region)#.to_numpy()
    distance_matrix[k,:] = distance[0]
    time_diff_matrix[k,:] = distance[1]
    lat_diff_matrix[k,:] = distance[2]
    lon_diff_matrix[k,:] = distance[3]


In [ ]:
np.sum(distance_matrix - distance_matrix.T)

In [ ]:
np.max(distance_matrix)

### Plotting matrix

In [ ]:
# plt.pcolormesh(distance_matrix, cmap='Reds')

####  Combine time and distance

In [ ]:
# Normalize distance using mean
distance_matrix_norm = distance_matrix/np.median(distance_matrix)

# Normalize time difference using mean
time_diff_matrix_norm = time_diff_matrix/np.median(time_diff_matrix)

In [ ]:
# # Establecer el tamaño de la figura
# plt.figure(figsize=(6, 6))  

# # Crear la visualización de la matriz utilizando pcolormesh
# plt.pcolormesh(distance_matrix_norm, cmap='Reds')
# plt.colorbar()
# plt.show()

In [ ]:
# Establecer el tamaño de la figura
plt.figure(figsize=(6, 6))  

# Crear la visualización de la matriz utilizando pcolormesh
plt.pcolormesh(time_diff_matrix_norm, cmap='Reds')
plt.colorbar()
plt.show()

In [ ]:
# Define time factor for exploration
time_factor = 0
cluster_selection_epsilon=0.0

# Mix both terms using time factor to weigth them
total_distance_matrix = distance_matrix_norm + time_factor*time_diff_matrix_norm

total_distance_matrix[np.abs(time_diff_matrix)>2*27] = 1e6
total_distance_matrix[np.abs(lon_diff_matrix)>30] = 1e6


In [ ]:
# plt.figure(figsize=(6, 6))
# plt.pcolormesh(total_distance_matrix, cmap='Reds')
# plt.colorbar()
# plt.show()

In [ ]:
clusterer = hdbscan.HDBSCAN(
    min_cluster_size=2,
    gen_min_span_tree=True,
    metric="precomputed",
    min_samples=2,
    cluster_selection_method="leaf",
    cluster_selection_epsilon=cluster_selection_epsilon
)
clusterer.fit(total_distance_matrix)

In [ ]:
# clusterer.condensed_tree_.plot()
# plt.show()

In [ ]:
cmap = cc.cm.glasbey_dark
color_palette = sns.color_palette(cmap.colors, np.max(clusterer.labels_)+1)

cluster_colors = [color_palette[x] if x >= 0
                  else (0, 0, 0)
                  for x in clusterer.labels_]

np.sum(clusterer.labels_ == -1)

In [ ]:
clusters_quantity = len(np.unique(clusterer.labels_))

len(allMag) // clusters_quantity

In [ ]:
allMag['cluster'] = clusterer.labels_

In [ ]:
duration = pd.Timedelta(8*365, "d") # In years
t1 = np.min(allMag['Date'])
t2 = np.max(allMag['Date'])

rotation_rate = 25.38 # in days

allMag['Rot_time'] = np.floor((allMag['Date']-np.min(allMag['Date'])).dt.days//rotation_rate)

# allMag['BMRLongitude'] -= 360*allMag['BMRLongitude']//360

ncycles = np.round((t2-t1)/duration)


# Size definitions
dpi = 400
pxx = 500  # Horizontal size of each panel
pxy = 10000  # Vertical size of each panel

nph = 5     # Number of horizontal panels
npv = 6     # Number of vertical panels 

# Padding
padv  = 0  #Vertical padding in pixels
padv2 = 0  #Vertical padding in pixels between panels
padh  = 0 #Horizontal padding in pixels at the edge of the figure
padh2 = 0  #Horizontal padding in pixels between panels

# Figure sizes in pixels
fszv = (npv*pxy + 2*padv + (npv-1)*padv2 )      #Vertical size of figure in pixels
fszh = (nph*pxx + 2*padh + (nph-1)*padh2 )      #Horizontal size of figure in pixels

# Conversion to relative units
ppxx   = pxx/fszh      # Horizontal size of each panel in relative units
ppxy   = pxy/fszv      # Vertical size of each panel in relative units
ppadv  = padv/fszv     #Vertical padding in relative units
ppadv2 = padv2/fszv    #Vertical padding in relative units
ppadh  = padh/fszh     #Horizontal padding the edge of the figure in relative units
ppadh2 = padh2/fszh    #Horizontal padding between panels in relative units


# Combine hemispheres 


## Start Figure
fig = plt.figure(figsize=(fszh/dpi,fszv/dpi), dpi = dpi)



size = (np.sqrt(allMag['BMRFlux']/1e21))
# size = np.abs(allMag['Tilt_rel']/5)

for i in np.arange(0,18):

    l1 = -45+i*5
    l2 = -45+(i+1)*5
        
    ax1 = fig.add_axes([ppadh+i*ppxx, ppadv, ppxx, ppxy])

    # mask = np.logical_and(allMag['Date']>=t1+n*duration, allMag['Date']<t1+(n+1)*duration)
    mask = allMag['BMRLatitude']>=l1
    mask = np.logical_and(mask, allMag['BMRLatitude']<l2)
    
    masknc =  np.logical_and(mask, clusterer.labels_ == -1)
    mask = np.logical_and(mask, clusterer.labels_ > -1)
    mask = np.logical_or(mask, np.in1d(clusterer.labels_, np.unique(clusterer.labels_[mask])))
    maskah = np.logical_and(mask, allMag['AntiHale'])
    
    ax1.scatter(allMag.loc[mask,'BMRLongitude'], allMag.loc[mask,'Date'], s=size[mask], alpha=0.8, ec='None', c=np.array(cluster_colors)[mask])
    ax1.scatter(allMag.loc[masknc,'BMRLongitude'], allMag.loc[masknc,'Date'], s=size[masknc], alpha=0.8, ec='None', c=np.array(cluster_colors)[masknc], marker="D")
    ax1.scatter(allMag.loc[maskah,'BMRLongitude'], allMag.loc[maskah,'Date'], s=size[maskah], alpha=0.8, ec='None', c='w', marker="*", lw=0.1)

    #mask = np.logical_and(mask, allMag['AntiHale'])
    #ax1.scatter(allMag.loc[mask,'Rot_time'], allMag.loc[mask,'BMRLongitude'], s=size[mask], alpha=1,fc='k', ec='None')

    # if n==0:
    ax1.set_title(f'{l1} to {l2}')

    ax1.set_ylim([np.min(allMag['Date']), np.max(allMag['Date'])])

     

In [ ]:
duration = pd.Timedelta(8*365, "d") # In years
t1 = np.min(allMag['Date'])
t2 = np.max(allMag['Date'])

rotation_rate = 25.38 # in days

allMag['Rot_time'] = np.floor((allMag['Date']-np.min(allMag['Date'])).dt.days//rotation_rate)

# allMag['BMRLongitude'] -= 360*allMag['BMRLongitude']//360

ncycles = np.round((t2-t1)/duration)


# Size definitions
dpi = 400
pxx = 500  # Horizontal size of each panel
pxy = 10000  # Vertical size of each panel

nph = 5     # Number of horizontal panels
npv = 6     # Number of vertical panels 

# Padding
padv  = 0  #Vertical padding in pixels
padv2 = 0  #Vertical padding in pixels between panels
padh  = 0 #Horizontal padding in pixels at the edge of the figure
padh2 = 0  #Horizontal padding in pixels between panels

# Figure sizes in pixels
fszv = (npv*pxy + 2*padv + (npv-1)*padv2 )      #Vertical size of figure in pixels
fszh = (nph*pxx + 2*padh + (nph-1)*padh2 )      #Horizontal size of figure in pixels

# Conversion to relative units
ppxx   = pxx/fszh      # Horizontal size of each panel in relative units
ppxy   = pxy/fszv      # Vertical size of each panel in relative units
ppadv  = padv/fszv     #Vertical padding in relative units
ppadv2 = padv2/fszv    #Vertical padding in relative units
ppadh  = padh/fszh     #Horizontal padding the edge of the figure in relative units
ppadh2 = padh2/fszh    #Horizontal padding between panels in relative units


# Combine hemispheres 


## Start Figure
fig = plt.figure(figsize=(fszh/dpi,fszv/dpi), dpi = dpi)

size = (np.sqrt(allMag['BMRFlux']/1e21))

lat_lim_range = np.arange(5, 25, 5)
lon_lim_range = np.arange(5, 25, 5)
cluster_selection_epsilon_range = np.arange(0.02, 0.22, 0.02)

for lat_lim in tqdm(lat_lim_range, position=0, total=lat_lim_range.shape[0], leave=True):
    for lon_lim in tqdm(lon_lim_range, position=1, total=lon_lim_range.shape[0], leave=False):
        for cluster_selection_epsilon in tqdm(cluster_selection_epsilon_range, position=2, total=cluster_selection_epsilon_range.shape[0], leave=False):
            total_distance_matrix = distance_matrix_norm + 0*time_diff_matrix_norm
            total_distance_matrix[np.abs(time_diff_matrix)>2*27] = 1e6        
            total_distance_matrix[np.abs(lon_diff_matrix)>lon_lim] = 1e6
            
            clusterer = hdbscan.HDBSCAN(
                min_cluster_size=2,
                gen_min_span_tree=True,
                metric="precomputed",
                min_samples=2,
                cluster_selection_method="leaf",
                cluster_selection_epsilon=float(cluster_selection_epsilon)
            )
            clusterer.fit(total_distance_matrix)

            cmap = cc.cm.glasbey_dark
            color_palette = sns.color_palette(cmap.colors, np.max(clusterer.labels_)+1)

            cluster_colors = [color_palette[x] if x >= 0
                            else (0, 0, 0)
                            for x in clusterer.labels_]
            

            for i in tqdm(np.arange(0,18), position=3, total=np.arange(0,18).shape[0], leave=False):

                l1 = -45+i*5
                l2 = -45+(i+1)*5
                    
                ax1 = fig.add_axes([ppadh+i*ppxx, ppadv, ppxx, ppxy])

                # mask = np.logical_and(allMag['Date']>=t1+n*duration, allMag['Date']<t1+(n+1)*duration)
                mask = allMag['BMRLatitude']>=l1
                mask = np.logical_and(mask, allMag['BMRLatitude']<l2)
                
                masknc =  np.logical_and(mask, clusterer.labels_ == -1)
                mask = np.logical_and(mask, clusterer.labels_ > -1)
                mask = np.logical_or(mask, np.in1d(clusterer.labels_, np.unique(clusterer.labels_[mask])))
                maskah = np.logical_and(mask, allMag['AntiHale'])
                
                ax1.scatter(allMag.loc[mask,'BMRLongitude'], allMag.loc[mask,'Date'], s=size[mask], alpha=0.8, ec='None', c=np.array(cluster_colors)[mask])
                ax1.scatter(allMag.loc[masknc,'BMRLongitude'], allMag.loc[masknc,'Date'], s=size[masknc], alpha=0.8, ec='None', c=np.array(cluster_colors)[masknc], marker="D")
                ax1.scatter(allMag.loc[maskah,'BMRLongitude'], allMag.loc[maskah,'Date'], s=size[maskah], alpha=0.8, ec='None', c='w', marker="*", lw=0.1)

                #mask = np.logical_and(mask, allMag['AntiHale'])
                #ax1.scatter(allMag.loc[mask,'Rot_time'], allMag.loc[mask,'BMRLongitude'], s=size[mask], alpha=1,fc='k', ec='None')

                # if n==0:
                ax1.set_title(f'{l1} to {l2}')

                ax1.set_ylim([np.min(allMag['Date']), np.max(allMag['Date'])])

            fig.savefig(f'figures/latl' + "{:.0f}".format(lat_lim) + 'lonl' + "{:.0f}".format(lon_lim) + '_ep' + "{:.2f}".format(cluster_selection_epsilon) + '_'+ str(i).zfill(4) + '.png', pad_inches=0, bbox_inches='tight') 
            fig.clf()                

### Plot only window of interest

In [ ]:
t1 = pd.Timestamp('2015-04-24T00')
t2 = pd.Timestamp('2015-08-11T00')

mask = np.logical_and(allMag[f'Date']>=t1, allMag[f'Date']<=t2)

allMag.loc[mask,:]

In [ ]:
duration = pd.Timedelta(8*365, "d") # In years

rotation_rate = 25.38 # in days

allMag['Rot_time'] = np.floor((allMag['Date']-np.min(allMag['Date'])).dt.days//rotation_rate)

# allMag['BMRLongitude'] -= 360*allMag['BMRLongitude']//360

ncycles = np.round((t2-t1)/duration)


# Size definitions
dpi = 400
pxx = 300  # Horizontal size of each panel
pxy = 2000  # Vertical size of each panel

nph = 5     # Number of horizontal panels
npv = 6     # Number of vertical panels 

# Padding
padv  = 0  #Vertical padding in pixels
padv2 = 0  #Vertical padding in pixels between panels
padh  = 0 #Horizontal padding in pixels at the edge of the figure
padh2 = 0  #Horizontal padding in pixels between panels

# Figure sizes in pixels
fszv = (npv*pxy + 2*padv + (npv-1)*padv2 )      #Vertical size of figure in pixels
fszh = (nph*pxx + 2*padh + (nph-1)*padh2 )      #Horizontal size of figure in pixels

# Conversion to relative units
ppxx   = pxx/fszh      # Horizontal size of each panel in relative units
ppxy   = pxy/fszv      # Vertical size of each panel in relative units
ppadv  = padv/fszv     #Vertical padding in relative units
ppadv2 = padv2/fszv    #Vertical padding in relative units
ppadh  = padh/fszh     #Horizontal padding the edge of the figure in relative units
ppadh2 = padh2/fszh    #Horizontal padding between panels in relative units


# Combine hemispheres 


## Start Figure
fig = plt.figure(figsize=(fszh/dpi,fszv/dpi), dpi = dpi)



size = (np.sqrt(allMag['BMRFlux']/1e21))
# size = np.abs(allMag['Tilt_rel']/5)

for i in np.arange(0,18):

    l1 = -45+i*5
    l2 = -45+(i+1)*5
        
    ax1 = fig.add_axes([ppadh+i*ppxx, ppadv, ppxx, ppxy])

    mask = np.logical_and(allMag[f'Date']>=t1, allMag[f'Date']<=t2)
    # mask = np.logical_and(allMag['Date']>=t1+n*duration, allMag['Date']<t1+(n+1)*duration)
    mask = np.logical_and(mask, allMag['BMRLatitude']>=l1)
    mask = np.logical_and(mask, allMag['BMRLatitude']<l2)
    
    masknc =  np.logical_and(mask, clusterer.labels_ == -1)
    mask = np.logical_and(mask, clusterer.labels_ > -1)
    mask = np.logical_or(mask, np.in1d(clusterer.labels_, np.unique(clusterer.labels_[mask])))
    maskah = np.logical_and(mask, allMag['AntiHale'])
    
    ax1.scatter(allMag.loc[mask,'BMRLongitude'], allMag.loc[mask,'Date'], s=size[mask], alpha=0.8, ec='None', c=np.array(cluster_colors)[mask])
    ax1.scatter(allMag.loc[masknc,'BMRLongitude'], allMag.loc[masknc,'Date'], s=size[masknc], alpha=0.8, ec='None', c=np.array(cluster_colors)[masknc], marker="D")
    ax1.scatter(allMag.loc[maskah,'BMRLongitude'], allMag.loc[maskah,'Date'], s=size[maskah], alpha=0.8, ec='None', c='w', marker="*", lw=0.1)

    #mask = np.logical_and(mask, allMag['AntiHale'])
    #ax1.scatter(allMag.loc[mask,'Rot_time'], allMag.loc[mask,'BMRLongitude'], s=size[mask], alpha=1,fc='k', ec='None')

    # if n==0:
    ax1.set_title(f'{l1} to {l2}')

    ax1.set_ylim([t1, t2])

    if i>0 and i<17:
        ax1.set_yticklabels([])
    
    if i == 17:
        ax1.yaxis.tick_right()
        ax1.yaxis.set_label_position("right")

     

In [ ]:
zarr_path = '/d0/euv/aia/preprocessed/aia_hmi_stacks_2010_2024_1d_full.zarr'

## Make movie

In [ ]:
movie_path = '/d0/amunozj/active_longitudes'
movie_path = f'{movie_path}/tf' + "{:.2f}".format(time_factor) + '_ep' + "{:.2f}".format(cluster_selection_epsilon) #+ 'plus'
if not os.path.exists(movie_path):
    os.makedirs(movie_path)
movie_path


In [ ]:
t1 = pd.Timestamp('2014-01-01T00')
t2 = pd.Timestamp('2016-01-01T00')
delta_days = 2*27

t_obs_movie = t_obs.loc[np.logical_and(t_obs>=t1, t_obs<=t2)]
closest_matches_inx = np.logical_and(t_obs>=t1, t_obs<=t2).to_numpy().nonzero()[0]

shape = (int(np.round(720*2*1.8)), 1440*2)
dpi = 200
fig = plt.figure(figsize=[shape[1]/2/dpi*1.1,shape[0]/2/dpi*1.1], constrained_layout=True, dpi=dpi)
spec = fig.add_gridspec(ncols=1, nrows=1, wspace=0, hspace=0)

for i, test_date in tqdm(enumerate(t_obs_movie), total=t_obs_movie.shape[0]):

    loaded_data = aia_hmi_stacks.aia_hmi.loc[test_date,channel, :, :].load()
    vmax = 1500
    vmin = -vmax

    header = {}
    for key in aia_hmi_header.keys():
        if key != '_ARRAY_DIMENSIONS':
            header[key] = aia_hmi_header[key][closest_matches_inx[i]]

    header['wavelenth'] = 6173.0
    header['telescop'] = 'SDO/HMI'
    header['instrume'] = 'HMI_COMBINED'
    hmimap = sunpy.map.Map(loaded_data.data, header)

    carr_header = make_heliographic_header(hmimap.date, hmimap.observer_coordinate, shape*2, frame='carrington')
    outmap = hmimap.reproject_to(carr_header)
    ax = fig.add_subplot(spec[0, 0], projection=outmap)
    outmap.plot(axes=ax, cmap=cmap, vmax=vmax, vmin=vmin)

    ## Add clustered regions
    visible_mask = np.logical_and(allMag['Date']>=test_date-pd.Timedelta(days=delta_days), allMag['Date']<=test_date+pd.Timedelta(days=delta_days))

    delta_date = allMag.loc[visible_mask, 'Date'].reset_index(drop=True) - test_date
    time_diff = ((delta_date.dt.total_seconds().values*u.s).to(u.d)).value
    corrected_longitude = allMag.loc[visible_mask, 'BMRLongitude'].reset_index(drop=True)
    corrected_longitude[corrected_longitude<0] = corrected_longitude[corrected_longitude<0]+360

    # print(allMag.loc[visible_mask, ['BMRLatitude', 'DRot']])

    corrected_longitude = allMag.loc[visible_mask, 'BMRLongitude'].reset_index(drop=True) - allMag.loc[visible_mask, 'DRot'].reset_index(drop=True) * time_diff * u.d    
    corrected_longitude = corrected_longitude - (np.abs(corrected_longitude)//360*np.sign(corrected_longitude))*360
    # corrected_longitude[corrected_longitude>180] = corrected_longitude[corrected_longitude>180] - 360
    # corrected_longitude[corrected_longitude<-180] = corrected_longitude[corrected_longitude>180] + 360

    cental_meridian_carr_lon = SkyCoord(0*u.deg, 0*u.deg, frame=frames.HeliographicStonyhurst).transform_to(frames.HeliographicCarrington(observer=hmimap.observer_coordinate, obstime=hmimap.date)).lon.value
    # print('central_lon', cental_meridian_carr_lon, i)

    for n, row in allMag.loc[visible_mask].reset_index(drop=True).iterrows():

        delta_lon = corrected_longitude[n] - cental_meridian_carr_lon
        if delta_lon < -180:
            delta_lon = 360+delta_lon
        if delta_lon > 180:
            delta_lon = -(360-delta_lon)

        bmr_center = SkyCoord(lat=row['BMRLatitude']*u.deg, lon=corrected_longitude[n]*u.deg, frame=HeliographicCarrington, obstime=hmimap.date, observer=hmimap.observer_coordinate)
        point_in_hpc = bmr_center.transform_to(hmimap.coordinate_frame)

        if row['cluster'] > -1:

            if np.abs(delta_lon)>=90 or np.abs(time_diff)[n]>10:
                alpha = (delta_days - (np.abs(time_diff[n])-10))/(delta_days)*0.4
                alpha = np.max([alpha,0]) + 0.1

                if time_diff[n] < 0:
                    ax.plot_coord(point_in_hpc, marker="v", ms=15, alpha=alpha, c=np.array(cluster_colors)[visible_mask][n], markeredgecolor='None')
                else:
                    ax.plot_coord(point_in_hpc, marker="^", ms=15, alpha=alpha, c=np.array(cluster_colors)[visible_mask][n], markeredgecolor='None')

            if np.abs(delta_lon)<=90 and np.abs(time_diff)[n]<10:
                ax.plot_coord(point_in_hpc, marker="o", ms=15, alpha=0.6, c=np.array(cluster_colors)[visible_mask][n], markeredgecolor='k')

        else:
            if np.abs(delta_lon)<=90 and np.abs(time_diff)[n]<10:
                ax.plot_coord(point_in_hpc, marker="s", ms=15, alpha=1, markerfacecolor='None', markeredgecolor='k')

    ax_lim = ax.get_ylim()
    ax1 = ax_lim[0] + (ax_lim[1]-ax_lim[0])*(50/180)
    ax2 = ax_lim[0] + (ax_lim[1]-ax_lim[0])*(130/180)
    ax.set_ylim(ax1,ax2)

    fig.savefig(f'{movie_path}/tf' + "{:.2f}".format(time_factor) + '_ep' + "{:.2f}".format(cluster_selection_epsilon) + '_'+ str(i).zfill(4) + '.png', pad_inches=0, bbox_inches='tight') 
    fig.clf()


In [ ]:
cr = []
for i, test_date in tqdm(enumerate(t_obs_movie), total=t_obs_movie.shape[0]):
    cental_meridian_carr_lon = SkyCoord(0*u.deg, 0*u.deg, frame=frames.HeliographicStonyhurst).transform_to(frames.HeliographicCarrington(observer='earth', obstime=test_date)).lon.value
    cr.append(cental_meridian_carr_lon)
 

In [ ]:
t_numpy = t_obs.to_numpy()

In [ ]:
t_dif = ((t_obs.iloc[1:].reset_index(drop=True) - t_obs.iloc[0:-1].reset_index(drop=True)).dt.total_seconds().values*u.s).to(u.d).value

In [ ]:
print(t_obs[(t_dif>1.5).nonzero()[0]])

In [ ]:
np.sum(t_dif>1.5)